In [ ]:
from source_files import *

import query_helpers

import math
import numpy as np
import pandas as pd

In [ ]:
SNOW_DEPTH_DIR = PurePath(f"{os.environ['HOME']}/project-data/ERW-Paper/1m")

aso_snow_depth = RasterFile(
    SNOW_DEPTH_DIR / '20180524_ASO_snow_depth_1m.tif',
    band_number=1
)
sfm_snow_depth = RasterFile(
    SNOW_DEPTH_DIR / '20180524_Agisoft_snow_depth_1m.tif',
    band_number=1
)

# 1 m

In [ ]:
df = db_query_to_df(
        'Select aso_snow_depth, sfm_snow_depth, sd_difference, casi_class, elevation FROM ERW_analysis'
)
df.loc[df['casi_class'] == 'Water', 'casi_class'] = 'Vegetation'

In [ ]:
df = query_helpers.diff_vol_swe(df, aso_snow_depth, sfm_snow_depth)

In [ ]:
positive_sfm = query_helpers.get_positive(df, 'sfm_snow_depth')
negative_sfm = query_helpers.get_negative(df, 'sfm_snow_depth')
no_values_sfm = query_helpers.get_no_data(df, 'sfm_snow_depth')

columns = ['casi_class', 'sfm_snow_depth', 'aso_snow_depth', 'sd_difference']

pd.set_option('display.float_format', lambda x: '%.2f m' % x)

# Snow Depth (1m)

In [ ]:
df[columns].agg([np.mean, np.median, np.std])

In [ ]:
df[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

### Positive SfM values

In [ ]:
positive_sfm[columns].agg([np.mean, np.median, np.std])

In [ ]:
positive_sfm[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

#### Depth between 3200 and 3500 m

In [ ]:
pd.set_option('display.float_format', lambda x: '{:.2%}'.format(x))

In [ ]:
low_elevation = positive_sfm[['sfm_snow_depth', 'aso_snow_depth', 'casi_class']].loc[df.elevation < 3200]

In [ ]:
mid_elevation = positive_sfm[['sfm_snow_depth', 'aso_snow_depth', 'casi_class']].loc[(df.elevation >= 3200) & (df.elevation <= 3500)]

In [ ]:
high_elevation = positive_sfm[['sfm_snow_depth', 'aso_snow_depth', 'casi_class']].loc[df.elevation > 3500]

In [ ]:
all_count = positive_sfm[['sfm_snow_depth', 'aso_snow_depth', 'casi_class']].groupby('casi_class').agg([np.size])

In [ ]:
low_elevation.groupby('casi_class').agg([np.size])/all_count

In [ ]:
mid_elevation.groupby('casi_class').agg([np.size])/all_count

In [ ]:
high_elevation.groupby('casi_class').agg([np.size])/all_count

### Negative SfM values

In [ ]:
negative_sfm[columns].agg([np.mean, np.median, np.std])

In [ ]:
negative_sfm[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

In [ ]:
pd.set_option('display.float_format', None)

# Snow Volume (1 m)

In [ ]:
m2_style = "{:,.0f} m<sup>3</sup>"

table_style = {
    'sfm_snow_volume': m2_style,
    'aso_snow_volume': m2_style,
    'difference': m2_style,
    'sfm % to aso': "{:.2%}",
    'percent_sfm_scene': "{:.2%}",
    'percent_aso_scene': "{:.2%}",
}
columns = ['casi_class', 'aso_snow_volume', 'sfm_snow_volume']

In [ ]:
total_volume = pd.DataFrame({
    'sfm_snow_volume': df.sfm_snow_volume.sum(),
    'aso_snow_volume': df.aso_snow_volume.sum(),
    },
    index=[0]
)
total_volume['difference'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume.style.format(table_style)

In [ ]:
captured_volume = pd.DataFrame({
    'sfm_snow_volume': positive_sfm.sfm_snow_volume.sum(),
    'aso_snow_volume': positive_sfm.aso_snow_volume.sum(),
    },
    index=[0]
)
captured_volume['difference'] = captured_volume.aso_snow_volume - captured_volume.sfm_snow_volume
captured_volume['sfm % to aso'] = captured_volume.sfm_snow_volume / captured_volume.aso_snow_volume
captured_volume.style.format(table_style)

In [ ]:
missed_volume = pd.DataFrame({
    'sfm_snow_volume': negative_sfm.sfm_snow_volume.sum(),
    'aso_snow_volume': negative_sfm.aso_snow_volume.sum(),
    },
    index=[0]
)
missed_volume['difference'] = missed_volume.aso_snow_volume - missed_volume.sfm_snow_volume
missed_volume['sfm % to aso'] = missed_volume.sfm_snow_volume / missed_volume.aso_snow_volume
missed_volume.style.format(table_style)

In [ ]:
assert (captured_volume.aso_snow_volume + missed_volume.aso_snow_volume).item(), total_volume.aso_snow_volume.item()

In [ ]:
num_pixels_aso = df.aso_snow_depth.count()
num_pixels_sfm = positive_sfm.aso_snow_depth.count()

In [ ]:
print("Pixels with depth:")
print(f"  ASO count: {num_pixels_aso:,}")
print(f"  SfM count: {num_pixels_sfm:,}")
print("Percent ASO pixels:")
print(f"  with values in SfM: {num_pixels_sfm/num_pixels_aso:.2%}")
print(f"  with negative in SfM: {negative_sfm.query('sfm_snow_depth == sfm_snow_depth').aso_snow_depth.count()/num_pixels_aso:.2%}")
print(f"  with no value in SfM: {no_values_sfm.aso_snow_depth.count()/num_pixels_aso:.2%}")

In [ ]:
pixel_stats_sfm = positive_sfm[columns].groupby('casi_class').count()
pixel_stats_aso = df[columns].groupby('casi_class').count()

grouped_volume = df[columns].groupby('casi_class').sum()
grouped_volume['difference'] = grouped_volume.aso_snow_volume - grouped_volume.sfm_snow_volume
grouped_volume['sfm % to aso'] = grouped_volume.sfm_snow_volume / grouped_volume.aso_snow_volume
grouped_volume['percent_sfm_scene'] = (pixel_stats_sfm.sfm_snow_volume / num_pixels_sfm)
grouped_volume['percent_aso_scene'] = (pixel_stats_aso.aso_snow_volume / num_pixels_aso)
grouped_volume.style.format(table_style)

### Overlapping area 

In [ ]:
grouped_volume = positive_sfm[columns].groupby('casi_class').sum()
grouped_volume['difference'] = grouped_volume.aso_snow_volume - grouped_volume.sfm_snow_volume
grouped_volume['sfm % to aso'] = grouped_volume.sfm_snow_volume / grouped_volume.aso_snow_volume
grouped_volume.style.format(table_style)

# SWE (1 m)

In [ ]:
m2_style = "{:,.0f} m"

table_style = {
    'sfm_swe': m2_style,
    'aso_swe': m2_style,
    'difference': m2_style,
    'sfm % to aso': "{:.2%}",
    'percent_sfm_scene': "{:.2%}",
}
columns = ['aso_swe', 'sfm_swe']

In [ ]:
total_swe = pd.DataFrame({
    'sfm_swe': df.sfm_swe.sum(),
    'aso_swe': df.aso_swe.sum(),
    },
    index=[0]
)
total_swe['difference'] = total_swe.aso_swe - total_swe.sfm_swe
total_swe['sfm % to aso'] = total_swe.sfm_swe / total_swe.aso_swe
total_swe.style.format(table_style)